In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [17]:
#Helper methods:

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    #return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def line_eq(X,slop,x1,y1):
    return slop * (X - x1) + y1

def x_in_line_eq (Y,slop, x1,y1):
    return ((-slop * x1) - Y + y1)/( -slop)

def get_slope(x1,y1,x2,y2):
    if x1==x2:
        return 1000
    else: 
        return (y2 - y1) / (x2 - x1)

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    
    right_line_x =[] 
    right_line_y =[]
    left_line_x=[] 
    left_line_y=[]
    slopes_left =[] 
    slopes_right =[]
    slope_threshold = 0.5
   
    if lines is None:
        return

    for segment in lines:
        for x1,y1,x2,y2 in segment:
            slope = get_slope(x1,y1,x2,y2)
            if (slope > 0 and slope > slope_threshold):
                    right_line_x.append((x1+x2)/2)
                    right_line_y.append((y1+y2)/2)
                    slopes_right.append(slope)
                   
            if (slope < 0 and abs(slope) > slope_threshold):
                    left_line_x.append((x1+x2)/2)
                    left_line_y.append((y1+y2)/2)
                    slopes_left.append(slope)
    
    shape = img.shape
    avr_slope_left = np.average(slopes_left)
    avr_slope_right = np.average(slopes_right)
    
   
    #left line
    if len(left_line_x) > 0 and len(left_line_y)>0:
         #end points calculation
            
        left_line_y.append(shape[1])
        left_line_x.append(x_in_line_eq (shape[1],avr_slope_left,left_line_x[0],left_line_y[0]))
        
        #draw line
        
        cv2.line(img, (int(min(left_line_x)), int(max(left_line_y))), 
             (int(max(left_line_x)), int(min(left_line_y))), color,10)
        
    #right line  
    
    if len(right_line_x) > 0 and len(right_line_y)>0:
        
         #end points calculation
            
        right_line_y.append(shape[1])
        right_line_x.append(x_in_line_eq (shape[1],avr_slope_right,right_line_x[0],right_line_y[0]))
        
        #draw line
        cv2.line(img, (int(min(right_line_x)), int(min(right_line_y))), 
             (int(max(right_line_x)), int(max(right_line_y))), color,10)
            

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)  
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [18]:
#Helper method for intial processing of the image
def grey_filter_img(image):
    grey = grayscale(image)
    return gaussian_blur(grey, 5)

In [19]:
#Helper method for reading an image file

def read_img(location):
    image = mpimg.imread(location)
    return image

In [20]:
#Helper method for processing of the image

def process_image (image):
    
    grey_image = grey_filter_img(image)
    
    low_threshold = 50
    high_threshold = 150
    edges = canny(grey_image, low_threshold, high_threshold)

    imshape = grey_image.shape
    vertices = np.array([[(80,imshape[0]),(480, 310), (470, 290), (imshape[1],imshape[0])]], dtype=np.int32)

    masked_edges = region_of_interest(edges, vertices)
    # Define the Hough transform parameters

    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments

    lines = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)

    # Draw the lines on the edge image
    lines_edges = weighted_img(lines, image, α=0.7, β=1., λ=0.)
    
    return lines_edges

In [21]:
#The pipeline method:

def test_pipeline():
    
    test_dir = "test_images/"
    list_img = os.listdir(test_dir)
    for img in list_img:
        location = os.path.join(test_dir,img)
        if os.path.isfile(location):
            test_image = process_image(read_img(location))
            mpimg.imsave(location, test_image)

In [22]:
test_pipeline()

In [23]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:04<00:00, 41.17it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 2.44 s, sys: 441 ms, total: 2.88 s
Wall time: 5.58 s


In [24]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [25]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:16<00:00, 41.84it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 7.83 s, sys: 1.89 s, total: 9.73 s
Wall time: 17 s


In [26]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [ ]:
Reflections

I imagine that my algorithm will be more robust if:
    1) Curves : I could draw a curvy line and not a straight line.
    2) Speed : I could process the images faster, because if the car 
        moves fast, the image processing might be inaccurate.
    3) Visibility : I could check an image of the road at 
        night /in a dusty place - the lines might not be so visible.